In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
df = pd.read_csv('streamer_df_clean')

In [6]:
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=2
    count = 0
    usernames = df.username.to_list()
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    chrome_driver_binary = "/Users/home/Documents/Bootcamp/chromedriver"
    dict = {}
    for user in usernames:
        count += 1
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"] for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        if count % 20 == 0:
            print(count)
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

# transfer stuff to csv
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/put_name', index=False)

In [7]:
socials = get_social(df)

<ipython-input-6-6117c8ec797a>:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440


In [31]:
socials.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/socials_df', index=False)

In [8]:
socials

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,https://twitter.com/tommyinnit,None,https://www.instagram.com/tommyinnitt/,https://discord.gg/an6rt4d
1,Quackity,https://twitter.com/quackity,"[http://youtube.com/c/Quackitwo, https://www.y...",https://www.instagram.com/quackity/?hl=en,https://www.discord.gg/quackity
2,Sapnap,https://twitter.com/sapnap,[https://www.youtube.com/channel/UCEKH2P285Qyl...,https://www.instagram.com/sapnapinsta/,https://discord.gg/qwmnrnp
3,auronplay,https://twitter.com/auronplay?ref_src=twsrc%5e...,[http://youtube.com/user/AuronPlay?sub_confirm...,https://www.instagram.com/auronplay/,None
4,xQcOW,https://twitter.com/xqc,[https://www.youtube.com/channel/UCmDTrq0LNgPo...,None,https://discord.gg/xqcow
...,...,...,...,...,...
2445,SwordsofLegendsOnline,https://twitter.com/playsolonline,[https://www.youtube.com/c/swordsoflegendsonli...,https://www.instagram.com/swordsoflegendsonline/,https://discord.gg/joinsolo
2446,easywithaces,https://twitter.com/easywithaces,[https://www.youtube.com/channel/UCce8Fo7_Va3N...,https://www.instagram.com/easywithaces,https://discord.gg/easywithaces
2447,SimpleFlips,http://twitter.com/simpleflips,"[https://youtube.com/simpleclips, http://youtu...",None,https://discord.gg/0ldoz0zjzyhrg48u
2448,Raptor,https://twitter.com/justraptah,"[https://www.youtube.com/user/EnergeticsTV, ht...",https://www.instagram.com/dominickraptor/,None


In [38]:
df_socs = socials.copy()
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,https://twitter.com/tommyinnit,None,https://www.instagram.com/tommyinnitt/,https://discord.gg/an6rt4d
1,Quackity,https://twitter.com/quackity,"[http://youtube.com/c/Quackitwo, https://www.y...",https://www.instagram.com/quackity/?hl=en,https://www.discord.gg/quackity
2,Sapnap,https://twitter.com/sapnap,[https://www.youtube.com/channel/UCEKH2P285Qyl...,https://www.instagram.com/sapnapinsta/,https://discord.gg/qwmnrnp
3,auronplay,https://twitter.com/auronplay?ref_src=twsrc%5e...,[http://youtube.com/user/AuronPlay?sub_confirm...,https://www.instagram.com/auronplay/,None
4,xQcOW,https://twitter.com/xqc,[https://www.youtube.com/channel/UCmDTrq0LNgPo...,None,https://discord.gg/xqcow
...,...,...,...,...,...
2445,SwordsofLegendsOnline,https://twitter.com/playsolonline,[https://www.youtube.com/c/swordsoflegendsonli...,https://www.instagram.com/swordsoflegendsonline/,https://discord.gg/joinsolo
2446,easywithaces,https://twitter.com/easywithaces,[https://www.youtube.com/channel/UCce8Fo7_Va3N...,https://www.instagram.com/easywithaces,https://discord.gg/easywithaces
2447,SimpleFlips,http://twitter.com/simpleflips,"[https://youtube.com/simpleclips, http://youtu...",None,https://discord.gg/0ldoz0zjzyhrg48u
2448,Raptor,https://twitter.com/justraptah,"[https://www.youtube.com/user/EnergeticsTV, ht...",https://www.instagram.com/dominickraptor/,None


In [39]:
df_socs['YouTube'][1]

['http://youtube.com/c/Quackitwo',
 'https://www.youtube.com/user/QuackityHQ',
 'https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA/',
 'https://www.youtube.com/c/Quackity']

In [61]:
test_link =  ['https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA?view=srkgjsef&loc=xmisland','https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8IM-FAKE?view=srkgjsef&loc=xmisland',  'https://www.youtube.com/channel/short_boi?view=srkgjsef&loc=xmisland']

In [34]:
def get_social_id(link):
    '''Extract streamer identifier at the end of their social media links'''
    if link[-1] == '/':
        link = link[:-1]
    start = link.rfind('/')
    id = link[start + 1:]

    return id

def get_yt_channel_id(links):
    '''Return the first genuine yt channel id in a list of yt links'''
    if not links:
        return None
    channel_id = None
    for link in links:
        link = link.split('?')[0]
        print(link)
        id = get_social_id(link)
        if len(id) == 24:
            channel_id = id
            break
    
    return channel_id


def parse_streamer_socials(socials_df):
    '''Return a dataframe with relevant ids per treamer and platform'''
    for (column_name, column_data) in socials_df.iteritems():
        if column_name != 'Discord':
#             if column_name == 'YouTube':
#                 socials_df[column_name] = socials_df[column_name].apply(get_yt_channel_id)
            socials_df[column_name] = column_data.apply(lambda x: get_social_id(x) if x else None)
            
        
    return socials_df

In [87]:
get_yt_channel_id(test_link)

https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA


'UCFbZ2e9IrPejOdp8wsKUxvA'

In [37]:
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,tommyinnit,None,tommyinnitt,https://discord.gg/an6rt4d
1,Quackity,quackity,None,?hl=en,https://www.discord.gg/quackity
2,Sapnap,sapnap,None,sapnapinsta,https://discord.gg/qwmnrnp
3,auronplay,auronplay?ref_src=twsrc%5egoogle%7ctwcamp%5ese...,None,auronplay,None
4,xQcOW,xqc,None,None,https://discord.gg/xqcow
...,...,...,...,...,...
2445,SwordsofLegendsOnline,playsolonline,None,swordsoflegendsonline,https://discord.gg/joinsolo
2446,easywithaces,easywithaces,None,easywithaces,https://discord.gg/easywithaces
2447,SimpleFlips,simpleflips,None,None,https://discord.gg/0ldoz0zjzyhrg48u
2448,Raptor,justraptah,None,dominickraptor,None


In [40]:
df_socs['YouTube'] = df_socs['YouTube'].apply(lambda x: get_yt_channel_id(x))

http://youtube.com/c/Quackitwo
https://www.youtube.com/user/QuackityHQ
https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA/
https://www.youtube.com/channel/UCEKH2P285Qylr60_e32Xn_g
http://youtube.com/user/AuronPlay
https://www.youtube.com/channel/UCFR2oaNj02WnXkOgLH0iqOA
https://www.youtube.com/channel/UCmDTrq0LNgPodDOFZiSbsww
https://www.youtube.com/channel/UCpq0Ou2ObQr-LbTB6hsl8BA
https://www.youtube.com/channel/UCaY_-ksFSQtTGk0y1HA_3YQ
https://www.youtube.com/tubbo
https://www.youtube.com/channel/UCAz5JW1_oryewk0eR-eP7Bw
https://youtube.com/lck
https://www.youtube.com/channel/UCw1DsweY9b2AKGjV4kGJP1A
https://www.youtube.com/channel/UCKQ-wNdh0kO5qnpPfXa2hjQ
https://www.youtube.com/rocketleagueesports
https://www.youtube.com/channel/UCnYjd7cPNluC7Hi9AZrBNMw
https://www.youtube.com/knossi
https://youtube.com/shroud
https://www.youtube.com/shroud
https://www.youtube.com/channel/UCExVswmCLmkkoBkjBp3Ta9Q
https://www.youtube.com/channel/UCDvm7YoLE5r3ZZ6MWyD2vGQ
https://www.youtube.com

https://www.youtube.com/c/PipePunk
https://www.youtube.com/subscription_center
https://www.youtube.com/subscription_center
https://www.youtube.com/channel/UCyHW5qfBXmmOR6mKVdAEOaQ
https://www.youtube.com/channel/UCv1tOT645xHOJrwdGlzjMaA
https://www.youtube.com/channel/UCjxnv4cOYHI-DSI61i7nniQ
https://www.youtube.com/channel/UCV-0J3NyC-sKw8p4q-vJYQg
https://www.youtube.com/channel/UCWsfGABc3RaJxKeE-VcYYGA
https://www.youtube.com/channel/UCxULoUHhi7STRs9dr7Lvm7Q
https://www.youtube.com/user/xTheSolution
https://www.youtube.com/channel/UC7wT7cZGlcECV4oXXqtLlrg
https://www.youtube.com/raiseyouredge
https://www.youtube.com/channel/UCUlMp_wTWqCH5ddSakUIKEQ
https://www.youtube.com/channel/UCNGq4mP3Ds5OUGjPo8IJOcw
https://youtube.com/kennzy
https://www.youtube.com/Kennzy
http://youtube.com/strippin
https://www.youtube.com/channel/UC9_W-Cia0ZuIdFBS3qCe1Aw
http://www.youtube.com/chocoTaco
https://www.youtube.com/channel/UCSri6c58uWro3kLTlcuFlVA
https://www.youtube.com/channel/UCrUmNy1Wi2b64GdvDr

https://www.youtube.com/channel/UCRRJnjs0QUOc7Hkp894ZOuQ
https://www.youtube.com/draineotvhq
https://www.youtube.com/channel/UC-6Wmm5BR5UJvFVSUztAitA
https://www.youtube.com/channel/UCxcQX3lLPT8ACwGKEQuwtDg
https://www.youtube.com/channel/UCKQt9M1KLrIuPNQGGKutWXw
https://www.youtube.com/smqcked
https://www.youtube.com/channel/UCC1U807bfxKSjcJJGBv89VQ
https://www.youtube.com/channel/UCQ5jGYuvu_dzEi2B1TpGZpA
https://www.youtube.com/weak3n
https://www.youtube.com/kurtslife
https://www.youtube.com/weak3ngames
http://www.youtube.com/subscription_center
https://www.youtube.com/user/l3ffen
https://youtube.com/l3ffen
https://www.youtube.com/c/chesscomes
https://www.youtube.com/channel/UCb8A85srkG5a0QUhB6ikVLg
https://www.youtube.com/user/AmazonVideoDE
https://youtube.com/c/Kinstaar92i
https://www.youtube.com/c/Kinstaar92i
https://www.youtube.com/channel/UCM53B7Vv9zQF5fBxOprb-KQ
https://www.youtube.com/kamolrf
https://www.youtube.com/channel/UChROEePD3Qsk5iOU_CGkl4w
https://www.youtube.com/chan

In [41]:
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,https://twitter.com/tommyinnit,None,https://www.instagram.com/tommyinnitt/,https://discord.gg/an6rt4d
1,Quackity,https://twitter.com/quackity,UCFbZ2e9IrPejOdp8wsKUxvA,https://www.instagram.com/quackity/?hl=en,https://www.discord.gg/quackity
2,Sapnap,https://twitter.com/sapnap,UCEKH2P285Qylr60_e32Xn_g,https://www.instagram.com/sapnapinsta/,https://discord.gg/qwmnrnp
3,auronplay,https://twitter.com/auronplay?ref_src=twsrc%5e...,UCFR2oaNj02WnXkOgLH0iqOA,https://www.instagram.com/auronplay/,None
4,xQcOW,https://twitter.com/xqc,UCmDTrq0LNgPodDOFZiSbsww,None,https://discord.gg/xqcow
...,...,...,...,...,...
2445,SwordsofLegendsOnline,https://twitter.com/playsolonline,None,https://www.instagram.com/swordsoflegendsonline/,https://discord.gg/joinsolo
2446,easywithaces,https://twitter.com/easywithaces,UCce8Fo7_Va3Np3wMhJLPQEA,https://www.instagram.com/easywithaces,https://discord.gg/easywithaces
2447,SimpleFlips,http://twitter.com/simpleflips,None,None,https://discord.gg/0ldoz0zjzyhrg48u
2448,Raptor,https://twitter.com/justraptah,UCV8Qdw3dsy2xtHTr7fiIVgw,https://www.instagram.com/dominickraptor/,None


In [42]:
clean_plattform_df = parse_streamer_socials(df_socs)

In [43]:
clean_plattform_df

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,tommyinnit,None,tommyinnitt,https://discord.gg/an6rt4d
1,Quackity,quackity,UCFbZ2e9IrPejOdp8wsKUxvA,?hl=en,https://www.discord.gg/quackity
2,Sapnap,sapnap,UCEKH2P285Qylr60_e32Xn_g,sapnapinsta,https://discord.gg/qwmnrnp
3,auronplay,auronplay?ref_src=twsrc%5egoogle%7ctwcamp%5ese...,UCFR2oaNj02WnXkOgLH0iqOA,auronplay,None
4,xQcOW,xqc,UCmDTrq0LNgPodDOFZiSbsww,None,https://discord.gg/xqcow
...,...,...,...,...,...
2445,SwordsofLegendsOnline,playsolonline,None,swordsoflegendsonline,https://discord.gg/joinsolo
2446,easywithaces,easywithaces,UCce8Fo7_Va3Np3wMhJLPQEA,easywithaces,https://discord.gg/easywithaces
2447,SimpleFlips,simpleflips,None,None,https://discord.gg/0ldoz0zjzyhrg48u
2448,Raptor,justraptah,UCV8Qdw3dsy2xtHTr7fiIVgw,dominickraptor,None


In [45]:
clean_plattform_df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/socials_nicks', index=False)

TODO: Insta ids need attention!!